# Table of Contents
* [Submitting HITs](#Submitting-HITs)
	* [Building URLs for images on s3](#Building-URLs-for-images-on-s3)
	* [submitting HITs in groups](#submitting-HITs-in-groups)
		* [creates HITs, careful with this one](#creates-HITs,-careful-with-this-one)
* [Reviewing latest HITs](#Reviewing-latest-HITs)
	* [download](#download)
	* [process](#process)
	* [clustering](#clustering)
		* [The two commands above interact with mechanical turk and can take a while](#The-two-commands-above-interact-with-mechanical-turk-and-can-take-a-while)
* [Merging latest round of HITs into combined dataset](#Merging-latest-round-of-HITs-into-combined-dataset)
	* [Load previously pickled results](#Load-previously-pickled-results)
	* [Load prior complete dataset if not in memory](#Load-prior-complete-dataset-if-not-in-memory)
	* [Updating full dataset](#Updating-full-dataset)
	* [Working with full dataset](#Working-with-full-dataset)
		* [worker specific](#worker-specific)
		* [Reviewing full dataset](#Reviewing-full-dataset)
		* [Sanitizing results- questions marked as answers](#Sanitizing-results--questions-marked-as-answers)
* [Worker analysis](#Worker-analysis)
	* [Basic worker stats](#Basic-worker-stats)
		* [HIT duration for pricing](#HIT-duration-for-pricing)
	* [Identifying high and low consensus workers](#Identifying-high-and-low-consensus-workers)
	* [Messaging workers](#Messaging-workers)
* [HIT end-of-life](#HIT-end-of-life)
	* [Pickle latest results](#Pickle-latest-results)
	* [Pickle combined dataset](#Pickle-combined-dataset)
	* [Accepting and deleting HITs... careful with these](#Accepting-and-deleting-HITs...-careful-with-these)
* [End](#End)


In [419]:
%%capture
from __future__ import division
import numpy as np
import pandas as pd
import scipy.stats as st
import itertools
import math
from collections import Counter, defaultdict
%load_ext autoreload
%autoreload 2

import matplotlib as mpl
mpl.use("Agg")
import matplotlib.pylab as plt
#%matplotlib notebook
%matplotlib inline
%load_ext base16_mplrc
%base16_mplrc light default
plt.rcParams['figure.figsize'] = (16.0, 10.0)

import re
from datetime import datetime
import dateutil.parser as dt_parse
import pickle
import boto
import json
import os

from copy import deepcopy
import boto.mturk.connection as tc
import boto.mturk.question as tq
from boto.mturk.qualification import PercentAssignmentsApprovedRequirement, Qualifications, Requirement

from keysTkingdom import mturk_ai2
from keysTkingdom import aws_tokes

import pdfextraction.amt_boto_modules as amt_util

# Submitting HITs

## Building URLs for images on s3

In [206]:
turk_data_flat_dir = '/Users/schwenk/wrk/stb/ai2-vision-textbook-dataset/diagrams/turk_data'

In [208]:
diagram_image_names = os.listdir(turk_data_flat_dir)

In [216]:
def make_hit_url(image_cat, image_name, task):
    task_urls = {
        'loc': "https://s3.amazonaws.com/ai2-vision-textbook-dataset/diagrams/annotation/loc/image_text_box_region_labeling.html?",
        'recog': "https://s3.amazonaws.com/ai2-vision-textbook-dataset/diagrams/annotation/recog/image_text_recognition.html?"
        }
    hit_url = task_urls[task] + 'category-image={},{}'.format(image_cat, image_name)
    return hit_url

In [217]:
task_type = 'loc'
hit_urls = [make_hit_url(task_type, image_name, task_type) for image_name in diagram_image_names]

In [255]:
len(hit_urls)

## submitting HITs in groups

In [227]:
## Switch between sandbox and the real world here ##
## DON'T FORGET to change submission POST request in the client ##

sandbox_host = 'mechanicalturk.sandbox.amazonaws.com' 
real_world_host = 'mechanicalturk.amazonaws.com'
mturk = tc.MTurkConnection(
    aws_access_key_id = aws_tokes.access_key,
    aws_secret_access_key = aws_tokes.access_secret_key,
    host = real_world_host,
    debug = 1 # debug = 2 prints out all requests.
)
current_account_balance = mturk.get_account_balance()[0]
print current_account_balance # a reminder of sandbox

In [231]:
static_params = {
    'title': "Locate Text in Science Diagrams",
    'description': "Draw boxes around text in an image",
    'keywords': ['image', 'science', 'text', 'bounding boxes' ],
    'frame_height': 800,
    'amount': 0.03,
    'duration': 3600 * 12,
    'lifetime': 3600 * 24 * 3,
    'max_assignments': 3
}

### creates HITs, careful with this one

In [236]:
images_to_submit = hit_urls[2:]
expected_cost = len(images_to_submit) *  static_params['amount'] * static_params['max_assignments']
if float(current_account_balance.amount) < expected_cost:
    print('WARNING -- account balance is too low -- WARNING')
print('expect this batch of HITs to cost: $' + str(expected_cost))

uncomment cell below only when ready to submit

In [237]:
# amt_util.create_hits_from_pages(mturk, images_to_submit, static_params)

# Reviewing latest HITs

## download

In [242]:
t_hit = r_hits_current_batch[0]

In [244]:
t_hit.HIT

In [245]:
r_hits_current_batch = amt_util.get_completed_hits(mturk)
assignment_results_current_batch = amt_util.get_assignments(mturk, r_hits_current_batch)

In [246]:
len(assignment_results_current_batch)

## process

In [248]:
assignment_results_current_batch.items()[:3]

In [249]:
print 'current assignment statuses'
print amt_util.get_assignment_statuses(assignment_results_current_batch)

In [253]:
submitted_assignments = defaultdict(list)
for hitid, assignments in assignment_results_current_batch.items():
    for assignment in assignments:
        if assignment.AssignmentStatus == 'Submitted':
            submitted_assignments[hitid].append(assignment)

In [254]:
len(submitted_assignments)

In [258]:
als = pd.Series([len(sas) for sas in submitted_assignments.values()])
als.value_counts()

In [ ]:
submitted_assignments

In [251]:
assignment_results_current_batch.values()[0][0]

In [116]:
def process_raw_hits(assignments_by_hit):
    mechanical_turk_results = defaultdict(list)
    for hit_id, hit_assignments in assignments_by_hit.items():
        for assignment in hit_assignments:
            for answers in assignment.answers:
                box_result = answers[2].fields[0]
                box_json = json.loads(box_result)
                mechanical_turk_results[hit_id].append({
                    assignment.AssignmentId: {answers[0].fields[0]: box_json}}
                )
    return mechanical_turk_results

In [259]:
raw_hit_results_current_batch = process_raw_hits(submitted_assignments)

In [118]:
test_box = raw_hit_results_current_batch.values()[0][0].values()[0].values()[0]
with open ('diagram_44.jpg.json', 'w') as f:
    box_json = json.dump(test_box, f)

In [474]:
def make_results_df(raw_hit_results):
    col_names = ['diagram', 'rectangle', 'hit_id', 'assignment_id']
    results_df = pd.DataFrame(columns=col_names)
    for hit_id, assignments in raw_hit_results.items():
        for assignment in assignments:
            for a_id, annotation in assignment.items():
                for diagram, rectangles in annotation.items():
                    for box in rectangles:
                        results_df.loc[len(results_df)] = \
                            [diagram, box, hit_id, a_id]
    return results_df

In [133]:
def make_consensus_df(results_df, no_consensus_flag):
    grouped_by_page = results_df.groupby(['diagram'])
    aggregated_df = grouped_by_page.agg(pd.DataFrame.mode)
#     aggregated_df.drop([ 'page', 'box_id', 'worker_id'], axis=1, inplace=True)
    aggregated_df = aggregated_df.fillna(no_consensus_flag)
    consensus_results_df = aggregated_df.reset_index()
    consensus_results_df.drop('level_2', axis=1, inplace=True)
    return consensus_results_df

In [475]:
results_df = make_results_df(raw_hit_results_current_batch)

In [477]:
results_df.head(2)

In [288]:
grouped_by_diagram = results_df.groupby(['diagram', 'assignment_id'])

In [290]:
diagram_agreement = defaultdict(list)
for page_user, boxes in grouped_by_diagram:
    diagram_agreement[page_user[0]].append(len(boxes))

In [497]:
agreement_series = []
diagrams_with_turker_agreement = []
for page, box_counts in diagram_agreement.items():
    agreement_series.append(len(set(box_counts)))
    if len(set(box_counts)) == 1:
        diagrams_with_turker_agreement.append(page)

In [294]:
pd.Series(agreement_series).value_counts()

In [1008]:
diff_series = []
two_turkers_agree = defaultdict(list)
for page, box_counts in diagram_agreement.items():
    set_list= list(set(box_counts))
    if len(set_list) == 2:
        off_by = abs(set_list[1] - set_list[0])
        diff_series.append(off_by)
        if off_by < 6:
            two_turkers_agree[str(off_by)].append(page)
        else:
            two_turkers_agree['lots'].append(page)
pd.Series(diff_series).value_counts()

In [1029]:
diff_series = []
no_turkers_agree = defaultdict(list)
for page, box_counts in diagram_agreement.items():
    set_list= list(set(box_counts))
    if len(set_list) == 3:
        off_by = abs(set_list[1] - set_list[0])
        diff_series.append(off_by)
        if off_by < 6:
            no_turkers_agree[str(off_by)].append(page)
        else:
            no_turkers_agree['lots'].append(page)
        diff_series.append(max(set_list) - min(set_list))
pd.Series(diff_series).value_counts()

In [298]:
(1419. + 433 + 52) / len(submitted_assignments)

## clustering

In [1027]:
def limit_rect(rect, max_x, max_y, border_pad = 2):
        if rect[0][0] < 0:
            rect[0][0] = border_pad
        if rect[0][1] < 0:
            rect[0][1] = border_pad
        if rect[1][0]  > max_x:
            rect[1][0] = max_x - border_pad
        if rect[1][1]  > max_y:
            rect[1][1] = max_y -border_pad
            
def draw_clusters(img_path, clustered_boxes, direction='rows'):
    
    def random_color():
        import random
        return random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)

    image = cv2.imread(img_path)
    max_height, max_width, channels = image.shape

    for idx, cluster in enumerate(clustered_boxes):
#         if len(cluster) < 3 or len(cluster) >3:
#             print idx
#             print cluster
#             print
#         else:
#             continue
        color = random_color()
        for box in cluster:
            limit_rect(box, max_width, max_height)
            cv2.rectangle(image, tuple(box[0]), tuple(box[1]), color=color, thickness=2)
#             font = cv2.FONT_HERSHEY_SIMPLEX
#             cv2.putText(image, str(idx) + ' : ' + str(len(cluster)), tuple(box[1]), font, 1,(0,0,0),2)
    return image

def draw_loc_clusters(diagram_locs, dest_image_dir = '/Users/schwenk/wrk/stb/check_clustering_km/', bracket=0):
    flattened_topic_dir = '/Users/schwenk/wrk/stb/ai2-vision-textbook-dataset/diagrams/turk_data/'
    for diagram_name, text_locs in diagram_locs.items():
        try:
            clustered_text_boxes = cluster_diagram_text_centers(text_locs, 3, bracket)
            image_path = flattened_topic_dir + diagram_name
            drawn_image = draw_clusters(image_path, clustered_text_boxes)
            cv2.imwrite(os.path.join(dest_image_dir, diagram_name), drawn_image)
        except OverflowError as e:
            print diagram_name, e

In [1019]:
import numpy as np
from sklearn.cluster import SpectralClustering
import cv2
import PIL.Image as Image
from sklearn.cluster import KMeans


def box_area(box):
    height = box[1][1] - box[0][1]
    width = box[1][0] - box[0][0]
    return height * width

def compute_intersection(b1, b2):
    dx = min(b1[1][0], b2[1][0]) - max(b1[0][0], b2[0][0])
    dy = min(b1[1][1], b2[1][1]) - max(b1[0][1], b2[0][1])
    if (dx >= 0) and (dy >= 0):
        intersection_area = dx * dy
        return intersection_area
    else:
        return

def comp_boxes_iou(b1, b2):
    b1 = b1.reshape(2, 2)
    b2 = b2.reshape(2, 2)
    b1_area = box_area(b1)
    b2_area = box_area(b2)
    intersection = compute_intersection(b1, b2)
    iou = intersection / (b1_area + b2_area - intersection)
    return iou

def comp_box_center(box):
    return [(box[1][0] + box[0][0]) / 2, (box[1][1] + box[0][1]) / 2]

In [1020]:
def cluster_diagram_text(text_rects, n_turkers_assigned = 3):
    n_clusters = int(len(text_rects) / n_turkers_assigned)
    text_clusterer = SpectralClustering(n_clusters, affinity=comp_boxes_iou, assign_labels='kmeans', eigen_solver='arpack', n_init=100)
    box_array = np.array(text_rects)
    box_array = np.array(text_rects)
    box_array_flattened = box_array.reshape(box_array.shape[0], 4)
    box_array_flattened = box_array_flattened.astype(np.float64)
    cluster_assignments = text_clusterer.fit_predict(box_array_flattened)
    clustered_boxes = [box_array[cluster_assignments == cluster_n].tolist() for cluster_n in range(n_clusters)]
    return clustered_boxes

In [1021]:
def cluster_diagram_text_centers(text_rects, n_turkers_assigned = 3, bracket=0):
    text_centers= [comp_box_center(rect) for rect in text_rects]
    remainder = len(text_centers) % n_turkers_assigned
    if not remainder:
        n_clusters = int(len(text_centers) / n_turkers_assigned)
    else:
        if bracket != 'lots':
            bracket_n = int(bracket)
        else:
            bracket_n = 7
        n_clusters = len(text_centers) // n_turkers_assigned + bracket
    text_clusterer = KMeans(n_clusters)
    box_array = np.array(text_centers)
    cluster_assignments = text_clusterer.fit_predict(box_array)
    clustered_boxes = [np.array(text_rects)[cluster_assignments == cluster_n].tolist() for cluster_n in range(n_clusters)]
    return clustered_boxes

In [1022]:
results_df_agreement = results_df[results_df['diagram'].isin(diagrams_with_turker_agreement)]

grouped_by_diagram = results_df_agreement.groupby('diagram')
diagram_locs = defaultdict(list)
for diagram_name, boxes in grouped_by_diagram:
    diagram_locs[diagram_name].extend(boxes['rectangle'].values)

In [974]:
draw_loc_clusters(diagram_locs)

In [1028]:
for bracket, diagrams_in_bracket in two_turkers_agree.items():
    out_dir = './two_turkers_agree_' + bracket 
    if not os.path.exists(out_dir):
        os.mkdir(out_dir)
    results_df_two_turkers_agree = results_df[results_df['diagram'].isin(diagrams_in_bracket)]
    grouped_by_diagram = results_df_two_turkers_agree.groupby('diagram')
    diagram_locs_almost = defaultdict(list)
    for diagram_name, boxes in grouped_by_diagram:
        diagram_locs_almost[diagram_name].extend(boxes['rectangle'].values)
    draw_loc_clusters(diagram_locs_almost, out_dir, bracket)

In [1030]:
for bracket, diagrams_in_bracket in no_turkers_agree.items():
    out_dir = './no_turkers_agree_' + bracket 
    if not os.path.exists(out_dir):
        os.mkdir(out_dir)
    results_df_two_turkers_agree = results_df[results_df['diagram'].isin(diagrams_in_bracket)]
    grouped_by_diagram = results_df_two_turkers_agree.groupby('diagram')
    diagram_locs_almost = defaultdict(list)
    for diagram_name, boxes in grouped_by_diagram:
        diagram_locs_almost[diagram_name].extend(boxes['rectangle'].values)
    draw_loc_clusters(diagram_locs_almost, out_dir, bracket)

In [1006]:
draw_loc_clusters(diagram_locs_almost, '/Users/schwenk/wrk/stb/check_clustering_km_almost_agree/')

## single test

In [1005]:
problem_name = 'aquifers_6523.png'
problem = diagram_locs_almost[problem_name]

image_path = flattened_topic_dir + problem_name
clustered_text_boxes_centers = cluster_diagram_text_centers(problem)

drawn_image = draw_clusters(image_path, clustered_text_boxes_centers)
Image.fromarray(drawn_image)

In [1004]:
[clust for clust in clustered_text_boxes_centers if len(clust) != 3]

### The two commands above interact with mechanical turk and can take a while

In [9]:
results_df_current_batch = amt_util.make_results_df(raw_hit_results_current_batch)
consensus_results_df_current_batch = amt_util.make_consensus_df(results_df_current_batch, 'No Consensus')

In [10]:
%%capture
no_consensus_hits = consensus_results_df_current_batch[consensus_results_df_current_batch['category'] == 'No Consensus']

In [102]:
flaw_rate = len(no_consensus_hits) / len(consensus_results_df_current_batch)
print 'text boxes without consensus comprise ' + '{0:0.2f}% '.format(flaw_rate * 100) + 'of the total'  

In [423]:
amt_util.write_results_df(consensus_results_df_current_batch)

In [419]:
# all pages from the latest batch
pages_to_review =pd.unique(consensus_results_df_current_batch['page'])

In [384]:
# all pages from the latest batch with a no-consensus box
pages_to_review =pd.unique(no_consensus_hits['page'])

In [424]:
sampling_rate = 1
sample_size = int(len(pages_to_review) * sampling_rate)
sampled_pages_to_review = list(np.random.choice(pages_to_review, size= sample_size, replace=False))
print 'sampling ' + str(sample_size) + ' pages out of ' + str(len(pages_to_review))
to_review = ['start_seq'] + sampled_pages_to_review

amt_util.review_results(to_review)
print 'posting to review tool, navigate to http://localhost:8080/ to see the sampled consensus results'

In [438]:
amt_util.count_pages_with_cat(combined_consensus_df, 'Question')

In [73]:
%%capture 
# all pages from the latest batch with a no-consensus box
suspect_subset = results_df_current_batch[results_df_current_batch['worker_id'].isin(suspect_workers[:5])]
amt_util.write_results_df(suspect_subset)
pages_to_review = pd.unique(suspect_subset['page'])

# Merging latest round of HITs into combined dataset 

## Load previously pickled results

In [383]:
batch_number = 7

with open('./store_hit_results_metadata/group_' + str(batch_number) + '/hit_info.pkl') as f:
    r_hits_previous_batch = pickle.load(f)
    
with open('./store_hit_results_metadata/group_' + str(batch_number) + '/assignment_info.pkl') as f:
    assignment_results_previous_batch = pickle.load(f)
    
# with open('./store_hit_results_metadata/group_' + str(batch_number) + '/raw_res.pkl') as f:
#     raw_hit_results_previous_batch = pickle.load(f)

In [397]:
# combined_results_batch = pd.read_pickle(data_pickled_dir + 'complete_df.pkl')
# combined_consensus_batch = pd.read_pickle(data_pickled_dir + 'consensus_df.pkl')

In [405]:
missing_hits = set(assignment_results_previous_batch.keys()).difference(assignment_results_current_batch.keys())
missing_hits_assignments = {k: v for k, v in assignment_results_previous_batch.items() if k in missing_hits}

## Load prior complete dataset if not in memory

In [2]:
data_pickled_dir = './store_hit_results_metadata/group_latest_combined/' 

In [3]:
combined_results_df = pd.read_pickle(data_pickled_dir + 'complete_df.pkl')
combined_consensus_df = pd.read_pickle(data_pickled_dir + 'consensus_df.pkl')
combined_consensus_with_workerid_df = pd.read_pickle(data_pickled_dir + 'consensus_df_w_workers.pkl') 

## Updating full dataset

In [426]:
combined_results_df = combined_results_df.append(results_df_previous_batch)
combined_consensus_df = combined_consensus_df.append(consensus_prev)
combined_consensus_with_workerid_df = combined_consensus_with_workerid_df.append(consensus_prevww)

In [4]:
%%capture
no_consensus_hits = combined_consensus_df[combined_consensus_df['category'] == 'No Consensus']

## Working with full dataset

### worker specific

In [961]:
flaw_rate = len(no_consensus_hits) / len(combined_consensus_df)
print 'text boxes without consensus are ' + '{0:0.2f}% '.format(flaw_rate * 100) + 'of the total'  

In [255]:
workers_to_qualify = worker_quality_df[worker_quality_df['submitted'] > 100].sort_values('flaw_ratio', ascending= True).head(25).index.tolist()

In [ ]:
workers_to_qualify

In [242]:
bad_and_prolific_workers[:15]

workers who contacted me

In [256]:
wrote_me = ['A3T8FDBJQV81FN',
'A3UUH3632AI3ZX',
'A2QVMCGDLTWV9',
'A356GXVAYWN0DV',
'A2V4PRG5UBNIX0',
'A333VJ2K6O6R79']

In [258]:
workers_to_ban = bad_and_prolific_workers[:5].index.tolist()
set(bad_and_prolific_workers.index).intersection(set(wrote_me))

In [275]:
workers_to_ban

In [260]:
combined_consensus_with_workerid_df.head()

After looking through the top few offenders, it's clear that 

1. the very worst intentionally submitted many blank pages

2. the rest didn't read the directions very closely

3. I'm comfortable rejecting the work of those with > 100 submissions

4. I'll ban the worst 15 from future HITs


### Reviewing full dataset

In [94]:
amt_util.write_results_df(combined_consensus_df, local_result_path)

In [4]:
# all pages from the complete
pages_to_review =pd.unique(combined_consensus_df['page'])

In [5]:
combined_consensus_df[combined_consensus_df['category'] == 'Question'].shape

In [6]:
pages_to_review =pd.unique(combined_consensus_df[combined_consensus_df['category'] == 'Question']['page'])

In [7]:
# all pages from the complete dataset with a no-consensus box
# pages_to_review =pd.unique(no_consensus_hits['page'])

In [8]:
sampling_rate = 0.30
sample_size = int(len(pages_to_review) * sampling_rate)
sampled_pages_to_review = list(np.random.choice(pages_to_review, size= sample_size, replace=False))
print 'sampling ' + str(sample_size) + ' pages out of ' + str(len(pages_to_review))
to_review = ['start_seq'] + sampled_pages_to_review

In [39]:
overmerged_dir = 'labeled-annotations/'
unmerged_dir = 'annotations_ws/'
lessmerged_dir = 'lessmerged-annotations/'
remerged_annotations  = 'test-remerged-annotations/'
anno_dir = lessmerged_dir
amt_util.review_results(to_review, anno_dir)
print 'posting to review tool, navigate to http://localhost:8080/ to see the sampled consensus results'

In [ ]:
# %%capture 
# # all pages from the latest batch with a no-consensus box

# # suspect_subset = combined_results_df[combined_results_df['worker_id'].isin(bad_and_prolific_to_review[10:15])]
# suspect_subset = combined_results_df[combined_results_df['worker_id'].isin(['A3VE5OH94HYHET'])]
# amt_util.write_results_df(suspect_subset)
# pages_to_review = pd.unique(suspect_subset['page'])

### Sanitizing results- questions marked as answers

In [67]:
question_only_df = combined_consensus_df[combined_consensus_df['category'] == 'Question']

In [68]:
pages_w_questions = pd.unique(question_only_df['page'])
len(pages_w_questions)

In [69]:
combined_df_question_pages_only = combined_consensus_df[combined_consensus_df['page'].isin(pages_w_questions)]

In [111]:
combined_df_question_pages_only.to_pickle('pages_w_questions.pkl')

In [70]:
question_pages_w_answers = pd.unique(combined_df_question_pages_only[combined_df_question_pages_only['category'] == 'Answer']['page'])

In [71]:
question_pages_w_answers.shape

In [72]:
incorrectly_marked_answers_df = combined_consensus_df[combined_consensus_df['page'].isin(question_pages_w_answers)]

In [73]:
pd.unique(incorrectly_marked_answers_df['page']).shape

In [83]:
to_correct_aq_df = combined_consensus_df[combined_consensus_df['page'].isin(incorrectly_marked_answers_df['page'])]

In [88]:
corrected_aq_df = to_correct_aq_df.replace('Answer', 'Question')

In [105]:
local_result_path='./ai2-vision-turk-data/textbook-annotation-test/corrected_raw_hit_results/'
# local_result_path='./ai2-vision-turk-data/textbook-annotation-test/corrected_unmerged/'

amt_util.write_results_df(corrected_aq_df, local_result_path)

In [106]:
pages_to_review = pd.unique(corrected_aq_df['page'])
sampling_rate = 1
sample_size = int(len(pages_to_review) * sampling_rate)
sampled_pages_to_review = list(np.random.choice(pages_to_review, size= sample_size, replace=False))
print 'sampling ' + str(sample_size) + ' pages out of ' + str(len(pages_to_review))
to_review = ['start_seq'] + sampled_pages_to_review

In [109]:
# anno_dir = 'corrected_unmerged/'
anno_dir = 'test-remerged-annotations/'

amt_util.review_results(to_review, anno_dir)
print 'posting to review tool, navigate to http://localhost:8080/ to see the sampled consensus results'

# Worker analysis

## Basic worker stats

In [118]:
print 'number of unique workers:'
pd.unique(combined_results_df['worker_id']).shape[0]

In [565]:
_ = combined_results_df['worker_id'].value_counts().hist(bins= 30)

### HIT duration for pricing

In [27]:
task_duration_seconds = []
for hit_id, assignments in assignment_results_current_batch.items():
    for assignment in assignments:
        hit_duration = dt_parse.parse(assignment.SubmitTime) - dt_parse.parse(assignment.AcceptTime)
        task_duration_seconds.append(hit_duration.seconds)

In [28]:
task_duration_series = pd.Series(task_duration_seconds)

In [872]:
_ = task_duration_series.hist(bins=30, log=True)

In [29]:
winsorized_durations = [t for t in task_duration_seconds if t < 300]
w_duration_series = pd.Series(winsorized_durations)

In [34]:
_ = pd.Series(w_duration_series).hist(bins=60)
plt.title('Worker task duration', fontsize=50, verticalalignment='bottom', color = b16_colors.b)
plt.ylabel("Number of Workers", fontsize=30, labelpad=10, color = b16_colors.b)
plt.xlabel("Seconds Spent on HIT", fontsize=30, labelpad=10, color = b16_colors.b)
plt.tick_params(axis='x', which='major', labelsize=20)
plt.tick_params(axis='y', which='major', labelsize=20)

In [30]:
print 'duration mode= ' + str(w_duration_series.mode().values[0])
print 'duration median= ' + str(w_duration_series.median())

## Identifying high and low consensus workers

In [113]:
consensus_with_workerid_df_this_batch = amt_util.make_consensus_df_w_worker_id(results_df_current_batch, consensus_results_df_current_batch)

In [138]:
len(pd.unique(worker_conflicts['worker_id']))

In [113]:
worker_conflicts = combined_consensus_with_workerid_df[combined_consensus_with_workerid_df['category'] != combined_consensus_with_workerid_df['consensus_category']]
all_worker_counts = combined_results_df['worker_id'].value_counts()
bad_worker_counts = worker_conflicts['worker_id'].value_counts()
worker_quality_df = pd.DataFrame([all_worker_counts, bad_worker_counts]).T
worker_quality_df.columns=['submitted', 'incorrect']
worker_quality_df['flaw_ratio'] = worker_quality_df['incorrect']/worker_quality_df['submitted']

good_workers = worker_quality_df.sort_values('flaw_ratio', ascending= True).index.tolist()

suspect_workers = worker_quality_df.sort_values('flaw_ratio', ascending= False).index.tolist()

worker_quality_df.sort_values('flaw_ratio', ascending= True).head(10)

I became concerned that I missed the perfect performers, but as I suspected perfect workers only did 1-2 HITs at most

In [147]:
aw_set = set(all_worker_counts.index)
bw_set = set(bad_worker_counts.index)
flawless_workers = list(aw_set.difference(bw_set))
all_worker_counts[all_worker_counts.index.isin(best_workers)]

In [156]:
bad_and_prolific_workers = worker_quality_df.sort_values('flaw_ratio', ascending= False).head(25).sort_values('incorrect', ascending= False)

In [176]:
bad_and_prolific_to_review = list(bad_and_prolific_workers[:15].index)

## Messaging workers

In [118]:
subject = "More science book annotation HITs are available"
message = """
Hello, 

If you're receiving this message you were among the top performers on the first group HITs I submitted.
I've submitted another group of HITs, with more to follow in the next few days. 
This task is slightly different from the first, so please review the new instructions before jumping in.

Happy to get any feedback you might have for the new HITs.
"""

In [119]:
_ = mturk.notify_workers(good_workers[:20], subject, message)

# HIT end-of-life

## Pickle latest results

In [95]:
#reset as needed
# gn = 6

In [375]:
gn

In [377]:
next_group = gn + 1
group_n = '_' + str(gn) + '/'

temp_store_dir = './store_hit_results_metadata/group'
try:
    os.mkdir(temp_store_dir + group_n)
except:
    OSError
    
result_file_name = 'hit_info.pkl'
assignment_file_name = 'assignment_info.pkl'
raw_results_file_name = 'raw_res.pkl'
complete_results_file = 'complete_df.pkl'
consensus_results_file = 'consensus_df.pkl'

amt_util.pickle_this(r_hits_current_batch, temp_store_dir + group_n + result_file_name)
amt_util.pickle_this(assignment_results_current_batch, temp_store_dir + group_n + assignment_file_name)
amt_util.pickle_this(raw_hit_results_current_batch, temp_store_dir + group_n + raw_results_file_name)
results_df_current_batch.to_pickle(temp_store_dir + group_n + complete_results_file)
consensus_results_df_current_batch.to_pickle(temp_store_dir + group_n + consensus_results_file)
print 'saved HIT batch number ' + str(gn)
print 'now onto batch ' +str(next_group) 
gn = next_group

## Pickle combined dataset

In [429]:
temp_store_dir = './store_hit_results_metadata/group'
group_n = '_latest_combined/'
try:
    os.mkdir(temp_store_dir + group_n)
except:
    OSError
    
complete_results_file = 'complete_df.pkl'
consensus_results_file = 'consensus_df.pkl'
consensus_results_file_w_workers = 'consensus_df_w_workers.pkl'

combined_results_df.to_pickle(temp_store_dir + group_n + complete_results_file)
combined_consensus_df.to_pickle(temp_store_dir + group_n + consensus_results_file)
combined_consensus_with_workerid_df.to_pickle(temp_store_dir + group_n + consensus_results_file_w_workers)

## Accepting and deleting HITs... careful with these

Uncomment only when ready to accept or delete hits

reject assignments carefully

In [321]:
# number_rejected_assignments, number_rejected_workers = amt_util.reject_assignments(mturk, workers_to_ban, combined_consensus_with_workerid_df)
# print 'rejecting ' + str(number_rejected_assignments) + ' assignments' + ' from ' + str(number_rejected_workers) + ' workers'

In [369]:
amt_util.get_assignment_statuses(assignment_results_current_batch_post_reject)

In [396]:
# amt_util.accept_hits(mturk, assignment_results_current_batch_post)

In [437]:
# amt_util.delete_some_hits(mturk, assignment_results_current_batch_post_)

In [226]:
# amt_util.delete_all_hits(mturk)

# End

In [ ]:
def area_intersect_over_union(segmentImage1, segmentImage2, segmentArea1 = 0, segmentArea2 = 0):
    areaIntersection = np.sum(segmentImage1 & segmentImage2)
    if areaIntersection == 0:
        return 0, 0, 0
    if segmentArea1 == 0:
        segmentArea1 = np.sum(segmentImage1)
    if segmentArea2 == 0:
        segmentArea2 = np.sum(segmentImage2)
    areaOverlap = areaIntersection / (segmentArea1 + segmentArea2 - areaIntersection + 0.01)
    areaSelfOverlap1 = areaIntersection / (segmentArea1 + 0.01)
    areaSelfOverlap2 = areaIntersection / (segmentArea2 + 0.01)
    return areaOverlap, areaSelfOverlap1, areaSelfOverlap2


def comp_iou(raw_image, ocr_box, anno_box):

    def update_img(mask, box):
        # box
        # some rectangle of the structure [[start_x, start_y], [end_x, end_y]
        start_x, start_y = box[0]
        end_x, end_y = box[1]
        mask[start_y:end_y + 1, start_x:end_x + 1] = True

    mask_a = np.zeros(raw_image.shape[0:2], dtype=np.bool)
    mask_b = np.zeros(raw_image.shape[0:2], dtype=np.bool)
    update_img(mask_a, ocr_box['rectangle'])
    update_img(mask_b, anno_box['rectangle'])

    areaOverlap, areaSelfOverlap1, areaSelfOverlap2 = areaIntersectOverUnion(mask_a, mask_b)

    #iou = (mask == 2).sum() / (mask >= 1).sum()
    return areaOverlap, areaSelfOverlap1, areaSelfOverlap2 

In [ ]:
problem_name = 'continental_drift_9076.png'
problem = diagram_locs[problem_name]


image_path = flattened_topic_dir + problem_name
drawn_image = draw_clusters(image_path, clustered_text_boxes_centers)
Image.fromarray(drawn_image)

[clust for clust in clustered_text_boxes_centers if len(clust) < 3]

len (clustered_text_boxes)

for box in clustered_text_boxes:
    print box
    print

In [ ]:
# def find_overlapping_boxes(user_boxes):
#     for user in range(1, len(user_boxes)):
#         for box1 in user_boxes[0]:
#             for box2 in sorted(user_boxes[user]):
#                 biou = comp_boxes_iou(box1, box2)
#                 if biou > 0.5
#                     print biou
# #         print